In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Shivani\OneDrive\Documents\B.Tech\IMU-Gait-Fall-Detection\imu-gait-fall-detection\data\processed\WISDM_combined.csv")

In [2]:
df['MEAN_Y'] = df[['Y0','Y1','Y2','Y3','Y4']].mean(axis=1)
df['VAR_Y'] = df[['Y0','Y1','Y2','Y3','Y4']].var(axis=1)
df['MEAN_Z'] = df[['Z0','Z1','Z2','Z3','Z4']].mean(axis=1)
df['VAR_Z'] = df[['Z0','Z1','Z2','Z3','Z4']].var(axis=1)

In [4]:
df['RESULTANT'] = np.sqrt(df['X0']**2 + df['Y0']**2 + df['Z0']**2)

In [5]:
from scipy.fft import fft

def dominant_freq(arr):
    f = np.abs(fft(arr))[:10]
    return np.argmax(f)

df['DOM_FREQ_X'] = df[['X0','X1','X2','X3','X4']].apply(dominant_freq, axis=1)
df['DOM_FREQ_Y'] = df[['Y0','Y1','Y2','Y3','Y4']].apply(dominant_freq, axis=1)
df['DOM_FREQ_Z'] = df[['Z0','Z1','Z2','Z3','Z4']].apply(dominant_freq, axis=1)

KeyError: 'ALIGNED'

In [6]:
from scipy.fft import fft
import numpy as np

def dominant_freq(row):
    arr = np.array(row.values, dtype=float)   # convert to numpy array
    f = np.abs(fft(arr))[:10]                 # take first 10 fft magnitudes
    return np.argmax(f)

df['DOM_FREQ_X'] = df[['X0','X1','X2','X3','X4']].apply(dominant_freq, axis=1)
df['DOM_FREQ_Y'] = df[['Y0','Y1','Y2','Y3','Y4']].apply(dominant_freq, axis=1)
df['DOM_FREQ_Z'] = df[['Z0','Z1','Z2','Z3','Z4']].apply(dominant_freq, axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = df.drop(['ACTIVITY'], axis=1)
y = df['ACTIVITY']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)   # keep 95% variance
X_pca = pca.fit_transform(X_scaled)

print("Original features:", X.shape[1])
print("Reduced features:", X_pca.shape[1])

Original features: 99
Reduced features: 31


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, stratify=y, random_state=42)

model_pca = RandomForestClassifier(n_estimators=200, random_state=42)
model_pca.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = model_pca.predict(X_test)
print("PCA-based RF accuracy:", accuracy_score(y_test, y_pred))

PCA-based RF accuracy: 0.75745672436751
